# Setup model

In [17]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4')

 ········


In [46]:
# test model using concept from previous langchain_basics notebook

from langchain_core.messages import SystemMessage,HumanMessage

messages = [
    SystemMessage(content="Explain the following mathematical concept."),
    HumanMessage(content="Fourier Series")
]

result = model.invoke(messages)

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parsed_output = parser.invoke(result)
print(parsed_output)

# using chain - prompt template - model - output parser

from langchain_core.prompts import ChatPromptTemplate

system_template = 'explain the following {topic} concept' 
human_template = '{text}'

prompt_template = ChatPromptTemplate.from_messages([
        ('system',system_template),
        ('user',human_template)
])

prompt = prompt_template.invoke({
    'topic':'science',
    'text':'metabolism'
})

prompt.to_messages()

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = prompt_template | model | parser
result = chain.invoke({
    'topic':'science',
    'text':'laminar flow'
})
result

A Fourier series is a mathematical tool used in analysis, specifically in the study of periodic functions. It is named after Jean-Baptiste Joseph Fourier, who introduced the concept in his study of heat transfer.

The Fourier series is a way to represent a function as a sum of simple sine and cosine waves. More specifically, it decomposes any periodic function or periodic signal into the sum of a set of simple oscillating functions, namely sines and cosines (or complex exponentials).

This concept is particularly useful in solving partial differential equations, in signal processing, and in data compression. It is also widely used in physics and engineering, as it helps to analyze and simplify complex systems with periodic behaviors.


'Laminar flow, also known as streamline flow, is a type of flow pattern of a fluid in which all the particles are flowing in parallel lines and in the same direction, without any disruption or cross currents. In this flow pattern, the fluid flows smoothly or in regular paths, in contrast to turbulent flow which is characterized by chaotic, irregular fluid motions.\n\nLaminar flow occurs at lower speeds and high viscosity where the fluid particles can flow without interruption. It\'s a fundamental concept in fluid dynamics and is used in various scientific and engineering applications, such as in the design of aircrafts and cars for smooth airflow, or in the medical field for blood flow through blood vessels. \n\nThe term "laminar flow" comes from the word "lamina", which means a layer. Hence, in laminar flow, the flow of fluid is imagined as layers sliding over one another. Each layer has a definite velocity and moves smoothly over the adjoining layer.'

# Build Chatbot

Based on the tutorial https://python.langchain.com/v0.2/docs/tutorials/chatbot/
you will need to continuously add the system's response to create a chatbot, else the chatbot wont rmb what has been mentioned before, thus there is a need to implement MessageHistory

get_session_history. This function is expected to take in a session_id and return a Message History object. This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain (we'll show how to do that)

In [55]:
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1443d99a0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1443d4e50>, root_client=<openai.OpenAI object at 0x14436b310>, root_async_client=<openai.AsyncOpenAI object at 0x1443d9a00>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')

In [57]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# create store
store  = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        # create new chat history
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# create a model with message history 
with_message_history = RunnableWithMessageHistory(model,get_session_history)
# - can pass in a chain with a model inside as well

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a session_id. This should look like:

In [61]:
config = {
    'configurable':{
        'session_id':'abc2'
    }
}

response = with_message_history.invoke(
    [
        HumanMessage(content='Whats my name?')
    ],
    config=config
)

response.content

"As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."

^ because havent provide the name

In [63]:
config = {
    'configurable':{
        'session_id':'abc2'
    }
}

response = with_message_history.invoke(
    [
        HumanMessage(content='Hi my name is dee!')
    ],
    config=config
)

response.content

'Nice to meet you, Dee! How can I assist you today?'

In [75]:
# check message history

store['abc2'].messages,store['abc2'].messages[0].content

([HumanMessage(content='Whats my name?'),
  AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 11, 'total_tokens': 49}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1c891908-3dec-49b1-a821-bacb977fb460-0', usage_metadata={'input_tokens': 11, 'output_tokens': 38, 'total_tokens': 49}),
  HumanMessage(content='Hi my name is dee!'),
  AIMessage(content='Nice to meet you, Dee! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 63, 'total_tokens': 77}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d70de965

In [77]:
config = {
    'configurable':{
        'session_id':'abc2'
    }
}

response = with_message_history.invoke(
    [
        HumanMessage(content='Whats my name now?')
    ],
    config=config
)

response.content

'Your name is Dee.'

In [85]:
# if use a different session_id name, then the chat refers to a new conversation with new message history

config2 = {
    'configurable':{
        'session_id':'abc4'
    }
}

response = with_message_history.invoke(
    HumanMessage("whats my name?"),
    config=config2
)

response.content

"As an artificial intelligence, I'm not able to know your name unless you tell me."

pass chain instead of model in RunnableWithMessageHistory

In [112]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage

prompts = ChatPromptTemplate.from_messages([
    ('system','You are a helpful personal assistant. Answer all questions to the best of your abilities.'),
    MessagesPlaceholder(variable_name='messages')
])

chain = prompt | model

In [114]:
response = chain.invoke(
    {
        'messages':[
            HumanMessage(content='hi! what is my name?')
        ]
    }
)

response.content

"I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."

now wrap this in the runnable message history

In [116]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)
config = {
    'configurable':{
        'session_id':'abc'
    }
}

response = with_message_history.invoke(
    [
        HumanMessage(content='hi whats my name?')
    ],
    config=config
)

response.content

'Your name is Alex. How can I assist you today?'

In [117]:
response = with_message_history.invoke(
    [
        HumanMessage(content='my name is alex')
    ],
    config=config
)

response.content

'Hello, Alex! How can I assist you today?'

In [118]:
response = with_message_history.invoke(
    [
        HumanMessage(content='whats my name?')
    ],
    config=config
)

response.content

'Your name is Alex. How can I help you today?'

In [129]:
# add language as input

prompt = ChatPromptTemplate.from_messages(
    [
        ('system','you are a helpful assistant. answer the questions in {language} please.'),
        MessagesPlaceholder(variable_name='messages')
    ]
)

chain = prompt | model

with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key='messages'
)

config = {
    'configurable':{
        'session_id':'abc6'
    }
}
response = with_message_history.invoke(
    {
        'messages':[HumanMessage(content='what is the solar system?')],
        'language': "Malay"
    },
    config=config
)

response.content

'Sistem suria adalah kumpulan planet, asteroid, meteor, komet dan benda angkasa lain yang mengorbit matahari. Ia termasuk planet seperti Merkurius, Venus, Bumi, Mars, Jupiter, Saturnus, Uranus, dan Neptunus.'

# Managing Conversation History

* there is a need to ensure that the history doesnt exceed the context window of the LLM
* BEFORE the prompt template but AFTER you load previous messages from Message History.

In [148]:
from langchain_core.messages import SystemMessage, AIMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)
# only keeps the last messages that fits the max 65 tokens + keep system message and start on human message

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [150]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."

In [152]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc10"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."